<a href="https://colab.research.google.com/github/jyulkm/Tyler-Tech-Practicum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
from openai import OpenAI
import requests
import torch

In [7]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [5]:
data = pd.read_csv('HHS.csv', usecols=['Name', 'Description', 'Tags'])

In [14]:
API_KEY = "sk-proj-kyXx1HK9QnXtLHzfx2nYT3BlbkFJKm558AgbaiWwF1rswtuN"

# Define the endpoint URL
API_ENDPOINT = "https://api.openai.com/v1/completions"

# Define the prompt for the model
prompt = "Once upon a time,"

# Define the parameters for the request
params = {
    "model": "gpt-3.5-turbo",  # Change this to the model you want to use
    "prompt": prompt,
    "max_tokens": 50  # Adjust the maximum number of tokens for the completion
}

# Define headers with your API key
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# Send a POST request to the API
response = requests.post(API_ENDPOINT, json=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Print the generated text
    print("Generated text:")
    print(response.json()["choices"][0]["text"])
else:
    # Print error message
    print("Error:", response.text)

Error: {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}



## Bart

In [19]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification

In [17]:
# Load pretrained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Description input
description = data['Description'][0]

# Tokenize and preprocess the description
input_ids = tokenizer(description, return_tensors="pt", max_length=1024, truncation=True)["input_ids"]

# Generate tags
output = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
generated_tags = tokenizer.decode(output[0], skip_special_tokens=True)

# Postprocess tags if needed
# For example, split the generated tags into a list
generated_tags_list = generated_tags.split(',')

# Display or use generated tags
print("Generated Tags:", generated_tags_list)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Generated Tags: ['The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention']


In [21]:
# Load pretrained BART model and tokenizer for sequence classification
model_name = "facebook/bart-large-mnli"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)

# Define text input for classification
text = data['Description'][0]

# Tokenize and preprocess the text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted label
predicted_label_id = torch.argmax(outputs.logits, dim=1).item()

# Map label id to label name (if available)
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}  # Example mapping for MNLI task
predicted_label_name = label_map[predicted_label_id]

# Print predicted label
print("Predicted Label:", predicted_label_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Predicted Label: neutral


In [16]:
data['Description'][0]

'<p>The healthcare-associated infection component of CDC’s <a href="https://www.cdc.gov/ncezid/dpei/eip/index.html" target="_blank">EIP</a> engages a <a href="https://www.cdc.gov/ncezid/dpei/eip/eip-sites.html" target="_blank">network</a> of state health departments and their academic medical center partners to help answer critical questions about emerging HAI threats, advanced infection tracking methods, and antibiotic resistance in the United States. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention. </p>'

In [18]:
generated_tags

'The healthcare-associated infection component of CDC’s EIP engages a network of state health departments and their academic medical center partners. Information gathered through this activity will play a key role in shaping future policies and recommendations targeting HAI prevention'